# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Цели: ###
1. Выяснить есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Выяснить есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Выяснить есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Выяснить как разные цели кредита влияют на его возврат в срок?



## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

bank = pd.read_csv('C:/Users/Nshan/Desktop/проект_Банк/datasets/data.csv')

bank.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В ходе предварительного осмотра данных замечены селудющие ошибки:
1. Пропуски в столбце доходов (возможны и в других столбцах, будем проверять)
2. Стаж работы показывает отрицательные показатели либо слишком большие положительные числа, что в теории невозможно
3. Значения столбца с образованием показывают проблемы с их значениями (разные регистры у слов) 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(bank.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [3]:
bank['days_employed'] = bank['days_employed'].fillna(bank['days_employed'].median())
bank['total_income'] = bank['total_income'].fillna(bank['total_income'].median())
bank.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [4]:
print(bank['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [5]:
bank['days_employed'] = bank['days_employed'].abs()
display(bank.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
print(bank['days_employed'].max())
print(bank['days_employed'].min())

401755.40047533
24.14163324048118


In [7]:
bank['days_employed'] = bank['days_employed'] / 24
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

1. Поскольку пропуски обнаружены в столбцах days_employed и total_income и их количество одинаковое, и их обладателями являются одни и те же люди, то можно сделать вывод, что эти люди официально не работают. Также возможно и то, что просто нет информации о месте их работы и, следовательно, доходах. Пока что будем считать, что их стаж работы и доходы будут равны 0. Для дальнейшего исследования имеет смысл присвоить им медианное значение, так как среди данных категорий нет людей, не получающих доходы (проверено, но в решение не занесено).
2. Слишком большое число в столбце days_employed нельзя объяснить тем, что человек одновременно работал на нескольких работах. Поэтому можно предположить, что система измерения в столбце days_employed указана в часах (при проверке данный гипотезы ччисла для максимального значения выглядят реалистично, учитывая тот факт, что человек мог работать на нескольких работах). Все значения подкорректированы относительно данной гипотезы. 
3. Отрицательные числа в столбце days_employed, возможно, возникли вследсвтие неправильного подсчёта (от дня начала работы отнимали текущий день). В результате чсила стали отрицательными. Привели их к нормальному виду (положительные), так как человек не может проработать -4 дня.
4. Также в столбце с детьми обнаружено 47 случаев, когда количесвто детей равно -47. Данная ошибка могла возникнуть предположительно в двух случаях: а) ошибка человеческого фактора, где случайно добавился минус, б) ошибка человеческого фактора, где в случае гибели ребёнка писали -1. Пока данный показатель менять не будем, чтобы не мешало четкому анаилуз. В процессе работы выясним, как влияет данный показатель на общую картину. 

### Замена типа данных

In [8]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
bank['days_employed'] = bank['days_employed'].astype('int')

In [10]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int32  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


**Вывод**

Поскольку человек не мог проработать дробное количество дней, тип колонки с информацией о стаже работы был изменён на int. Для этого был использован метод astype(), так как to_numeric() превращает данные во float

### Обработка дубликатов

In [11]:
print(bank.duplicated().sum())
bank[bank.duplicated()].head()

54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,50,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,покупка жилья для семьи
4182,1,50,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
4851,0,50,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,свадьба
5557,0,50,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,сыграть свадьбу
7808,0,50,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,на проведение свадьбы


In [12]:
bank['education'] = bank['education'].str.lower()
bank['family_status'] = bank['family_status'].str.lower()
bank['income_type'] = bank['income_type'].str.lower()
bank['purpose'] = bank['purpose'].str.lower()

In [13]:
bank.duplicated().sum()

71

In [14]:
bank = bank.drop_duplicates().reset_index(drop=True)

In [15]:
bank.duplicated().sum()

0

**Вывод**

В качестве метода для поиска дубликатов был выбран duplicated(), так как в отличие от count_values() он обрабатывает всю таблицу, а не Series (конкретный столбец). Совпадение данных при анализе одного столбца не позволяют нам сделать вывод, что это дубликат.
Сами же дубликаты могли возникнуть в связи с тем, что человек мог оставить сразу несколько заявок на получение кредита.

### Лемматизация

In [16]:
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,38,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,119,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,6,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,288,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,91,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [17]:
m = Mystem()

In [18]:
purpose_list = list(bank['purpose'])
#print(purpose_list)
purpose_str = ' '.join(purpose_list)
#print(purpose_str)
lemmas = m.lemmatize(purpose_str)
print(Counter(lemmas))
print(bank['purpose'].value_counts())

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})
свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                              

**Вывод**

1. Леммы были выделены на основании целей кредита. Поскольку функция lemmatize() берёт в качестве аргумента строки, мы превратили Series с целями в список, а затем в строку.
2. При анализе Лемм можно сделать вывод, что основной целью кредитов является покупка недвижимости, затем на образование и на свадьбу.

### Категоризация данных

In [ ]:
#list_unique_purpose = ['cвадьба', 'жилье', 'недвижимость', 'автомобиль', 'образование']

def change_purpose(purpose):
    if 'жилье' in m.lemmatize(purpose) or 'недвижимость' in m.lemmatize(purpose):
        return 'недвижимость'
    elif 'свадьба' in m.lemmatize(purpose):
        return 'свадьба'
    elif 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    elif 'образование' in m.lemmatize(purpose):
        return 'образование'
    else:
        return 'другие цели'


bank['purpose_group'] = bank['purpose'].apply(change_purpose)


display(bank.head(10))

print(bank['purpose_group'].value_counts())

In [ ]:
def change_total_income(total_income):
    if total_income <= 30000:
        return 'минимальный доход'
    elif total_income > 30000 and total_income <= 60000:
        return 'низкий доход'
    elif total_income > 60000 and total_income <= 120000:
        return 'средний доход'
    else:
        return 'высокий доход'



bank['total_income_group'] = bank['total_income'].apply(change_total_income)

#display(bank.head(10))
print(bank['total_income_group'].value_counts()) 

In [ ]:
print(bank['income_type'].value_counts())
print(bank['education'].value_counts())

In [ ]:
children_debt_rel = bank[['children', 'debt']]
family_status_debt_rel = bank[['family_status', 'debt']]
total_icnome_debt_rel = bank[['total_income_group', 'debt']]
purpose_group_debt_rel = bank[['purpose_group', 'debt']]

**Вывод**

1. После лемматизации было ясно, что цели кредитов слишком часто повоторяют один и тот же смысл. Для этого цели бы сгруппированы в 5 групп.
2. В связи с тем, что показатели колонок имеют слишком много уникальных значений, для анализа будет целесообразно их сгруппировать в 4 категории: необходимый доход (до 30 000 руб), низкий доход (до 60 000), средний доход (до 120 000), высокий доход (более 150 000). Данные цифры подобраны исходя из среднего количества детей в семье и прожиточного минимума в Российской Федерации.
3. Далее создаются компактно сгрупирированные таблицы для наглядного отображения результатов анализа и для более высокой скорости обработки данных. Возможно также создание таблиц словарей и таблиц с данными, в данном случае для наглядность решено не использовать этот способ.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_debt_rel = children_debt_rel.groupby('children').agg({'debt': ['sum', 'count']})
children_debt_rel ['ratio'] = children_debt_rel['debt']['sum']/children_debt_rel['debt']['count']
display(children_debt_rel.sort_values(by='ratio'))

**Вывод**

Чёткой зависимости между наличием детей и возвратом кредита в срок нет. Количесвто детей -1 картины не поменяют, если их добавить к столбцу с количеством детей 0 или 1. Доля семей у которых 5, 0 или 3 ребёнка показатели наиболее высокие показатели возврата кредитов в срок (в семье, где 3 и более ребёнка - многодетная семья. Предусмотрены льготы).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_debt_rel = bank[['family_status', 'debt']]
family_status_debt_rel = family_status_debt_rel.groupby('family_status').agg({'debt': ['sum', 'count']})
family_status_debt_rel['ratio'] = family_status_debt_rel['debt']['sum']/family_status_debt_rel['debt']['count']
display(family_status_debt_rel.sort_values('ratio'))

**Вывод**

Кредиты в срок чаще возвращают люди без супруга/супруги, ранее состоявших в браке. Женатые/замужние люди тоже часто возвращают кредиты в срок, а вот не женатые люди и пары в гражданском браке возвращают кредит в срок реже. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
total_icnome_debt_rel = total_icnome_debt_rel.groupby('total_income_group').agg({'debt': ['sum', 'count']})
total_icnome_debt_rel['ratio'] = total_icnome_debt_rel['debt']['sum']/total_icnome_debt_rel['debt']['count']
display(total_icnome_debt_rel.sort_values('ratio'))

**Вывод**

Прямой зависимостьи между уровнем дохода и возвратом кредита в срок нет. Люди с высоким заработком показывают лучшие показатели, что логично. Удивительно, что люди со средним заработком (60к-120к) показывают худшие показатели. Возможно это связано с нестабильным заработком или большим объёмом кредитных средств.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_group_debt_rel = purpose_group_debt_rel.groupby('purpose_group').agg({'debt': ['sum', 'count']})
purpose_group_debt_rel['ratio'] = purpose_group_debt_rel['debt']['sum']/purpose_group_debt_rel['debt']['count']
display(purpose_group_debt_rel.sort_values('ratio'))

**Вывод**

Кредиты на недвижимое имущество возвращают в срок чаще, что объясняется следующими особенностями: данные кредиты оформляются на большие суммы (при несоблюдении сроков по платежам кредиты серьёзным образом увеличиваются); при неуплате данных кредитов банки имущество могут забрать.
Наименее эффективны для банка кредиты для покупки автомобиля.

## Шаг 4. Общий вывод

В ходе работы были проанализированы и прокомментированы 4 вопроса: 
* Есть ли зависимость между наличием детей и возвратом кредита в срок? 
Чёткой зависимости между наличием детей и возвратом кредита в срок нет. Доля семей у которых 5, 0 или 3 ребёнка показатели наиболее высокие показатели возврата кредитов в срок.
* Есть ли зависимость между семейным положением и возвратом кредита в срок? 
Кредиты в срок чаще возвращают люди без супруга/супруги, ранее состоявших в браке. 
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
Прямой зависимостьи между уровнем дохода и возвратом кредита в срок нет. Люди с высоким заработком показывают лучшие показатели. Люди со средним заработком (60к-120к) показывают худшие показатели. 
* Как разные цели кредита влияют на его возврат в срок?
Кредиты на недвижимое имущество возвращают в срок чаще.


В результате можно сделать вывод, что наиболее безопасны, с точки зрения возврата кредита в срок, для банка кредиторы, которые:
1. имеют детей (если в семье 3 ребенка) либо семьи без детей;
2. ранее состояли в брачных отношениях, но сейчас одинокие;
3. имеют высокие доходы (от 180 000 рублей);
4. берут кредит на покупку жилья или другого недвижимого имущества.